# Theory

Let's call a given natal kick distribution model $M(\vec{\theta})$, where $\vec{\theta} = \{v_{ns}, \sigma_{ns}\}$ for the Mandel Muller 2020 model.

Any given pulsar data set from Deller can be expressed as a collection of equally likely data points $\{d_i\}$.

Then, the likelihood of reproducing the data set using our model is given by:
$$p(d_i | M) = \int p(d_i | v_i) \cdot p(v_i|M) \cdot d v_i$$

Using the data files from Deller, we can compute the posterior on the velocity distribution as: 
$$p(v_i | d_i) = \frac{p(d_i | v_i) \cdot \pi(v_i)}{p(d_i)}$$
where $\pi(v_i)$ is our prior on the velocity distribution, and $p(d_i)$ is a normalization.


We will assume a flat prior $\pi$, and we will ignore the normalization factor $p(d_i)$ since it doesn't affect the probability distribution, i.e. it is independent of $v_i$.

Therefore, we can make the simplifying assumption that 
$$p(v_i|d_i) = p(d_i|v_i).$$

We can now re-write the likelihood equation as:
$$p(d_i | M) \approx \int p(v_i | d_i) \cdot p(v_i|M) \cdot d v_i .$$

$p(v_i | d_i)$ can be read off from the posterior data, since it is simply the probability distribution of the velocity measurements.

Thus, the probability of drawing a given pulsar $d_i$ from a model $M$ is given by:
$$p(d_i|M) = \langle p(v_i | M) \rangle.$$

Here, $p(v_i|M)$ is the probability of drawing a given velocity, which appears in the data set, from model $M$. The average over all these probabilities gives the overall probability of drawing this pulsar from the model.

Finally, the probability of drawing all $N$ pulsars from model M is
$$p(d|M) = \prod_{i=1}^{N} p(d_i|M).$$

(RESOLVED) To get $p(v_i|M)$, for now we will simply assume that the model velocities are transverse. **In general, this is a poor assumption**. We must project all the velocities along isotropically distributed planar directions to get a modeled probability distribution for Transverse Velocity. I expect that the current calculation **underestimates** the true velocity multiplier.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import time
from scipy import interpolate
import os

In [2]:
from natal_kick_tools import mandel_muller_likelihood_functions as mmf

# Probability from COMPAS simulated models

In [3]:
# Define the models of interest
bh_kicks=[200]
ns_kicks_sse = [300, 350, 400, 450, 500, 550, 600, 650, 700]
ns_kicks_bse=[400, 450, 500, 550, 600, 650]
sigmas = [0.1, 0.2, 0.3, 0.4, 0.5]

# Define the location of the posterior data
pulsar_data_dir = "../correctedPulsarPosteriors/correctedVtData"

# Generate 2D Projected velocities

In [ ]:
# SSE
mmf.v3d_to_v2d(bh_kicks=bh_kicks_sse, ns_kicks=ns_kicks_sse, sigmas=sigmas, mode='sse', \
               work_dir = os.environ['WORK'] + f'/supernova_remnant', output_dir='model_velocities')


In [4]:
# BSE
mmf.v3d_to_v2d(bh_kicks=bh_kicks, ns_kicks=ns_kicks_bse, sigmas=sigmas, mode='bse', \
               work_dir = os.environ['WORK'] + f'/supernova_remnant_bse', output_dir='model_velocities_bse')

Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.1_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.2_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.3_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.4_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_400_sigma_0.5_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.1_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.2_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant_bse/bh_200_ns_450_sigma_0.3_combined.h5


# Compute all the likelihoods for the 89 pulsars

In [4]:
# SSE
start = time.time() 
p_models_sse = mmf.get_pulsar_probability(pulsar_data_dir, model_data_dir='model_velocities',\
                                      bh_kicks=bh_kicks, ns_kicks=ns_kicks_sse, sigmas=sigmas,\
                                      output_dir='calculatedModelLikelihoods')
end = time.time()

print("Complete calculation completed in:", end - start, "s")

Loading projected model data from model_velocities/vns_300_sigma_0.1_velocities
Loading projected model data from model_velocities/vns_300_sigma_0.2_velocities
Loading projected model data from model_velocities/vns_300_sigma_0.3_velocities
Loading projected model data from model_velocities/vns_300_sigma_0.4_velocities
Loading projected model data from model_velocities/vns_300_sigma_0.5_velocities
Loading projected model data from model_velocities/vns_350_sigma_0.1_velocities
Loading projected model data from model_velocities/vns_350_sigma_0.2_velocities
Loading projected model data from model_velocities/vns_350_sigma_0.3_velocities
Loading projected model data from model_velocities/vns_350_sigma_0.4_velocities
Loading projected model data from model_velocities/vns_350_sigma_0.5_velocities
Loading projected model data from model_velocities/vns_400_sigma_0.1_velocities
Loading projected model data from model_velocities/vns_400_sigma_0.2_velocities
Loading projected model data from model_

Likelihood calculation for vns_700_sigma_0.5 completed in: 0.26038599014282227 s
Calculation Complete!

Complete calculation completed in: 71.49700856208801 s


In [5]:
# BSE
start = time.time() 
p_models_bse = mmf.get_pulsar_probability(pulsar_data_dir, model_data_dir='model_velocities_bse',\
                                      bh_kicks=bh_kicks, ns_kicks=ns_kicks_bse, sigmas=sigmas,\
                                      output_dir='calculatedModelLikelihoods_bse')
end = time.time()

print("Complete calculation completed in:", end - start, "s")

Loading projected model data from model_velocities_bse/vns_400_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.2_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.3_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.4_velocities
Loading projected model data from model_velocities_bse/vns_400_sigma_0.5_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.2_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.3_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.4_velocities
Loading projected model data from model_velocities_bse/vns_450_sigma_0.5_velocities
Loading projected model data from model_velocities_bse/vns_500_sigma_0.1_velocities
Loading projected model data from model_velocities_bse/vns_500_sigma_0.2_vel

# Examine Output

In [5]:
p_models_sse = p_models_sse/np.max(p_models_sse)
p_models_2d_sse = p_models_sse.reshape([len(ns_kicks_sse), len(sigmas)])
print(np.array2string(p_models_2d_sse, formatter={'float_kind': '{0:.3f}'.format}))

[[0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.001 0.001 0.001]
 [0.000 0.004 0.029 0.060 0.022]
 [0.002 0.046 0.334 0.503 0.149]
 [0.004 0.096 0.696 1.000 0.203]
 [0.003 0.046 0.384 0.573 0.127]
 [0.001 0.014 0.154 0.206 0.039]
 [0.000 0.002 0.025 0.044 0.009]
 [0.000 0.000 0.002 0.006 0.002]]


In [6]:
p_models_bse = p_models_bse/np.max(p_models_bse)
p_models_2d_bse = p_models_bse.reshape([len(ns_kicks_bse), len(sigmas)])
print(np.array2string(p_models_2d_bse, formatter={'float_kind': '{0:.3f}'.format}))

[[0.000 0.011 0.062 0.077 0.036]
 [0.004 0.079 0.350 0.508 0.179]
 [0.010 0.145 0.733 1.000 0.383]
 [0.008 0.106 0.582 0.779 0.213]
 [0.002 0.032 0.201 0.270 0.080]
 [0.000 0.004 0.033 0.060 0.018]]
